In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import uniform

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [17]:
data = pickle.load(open("/content/drive/My Drive/Kowshik_Srijan_NER_without_vocab/Data/tfidf_pca_result_final_500.pickle", "rb"))
X_pred = (pd.read_pickle("/content/drive/My Drive/Kowshik_Srijan_NER_without_vocab/Data/GS_tfidf_pca_result_final_500.pickle")).drop(columns = ['doc_id'])

gc.collect()

0

In [18]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,doc_id,importance
0,0.000112,0.000680,0.000545,0.001455,0.001007,0.000400,0.000728,0.005343,-0.000395,0.241540,-0.118889,-0.004854,-0.015600,0.005033,-0.002511,0.001477,-0.000665,-0.006386,-0.002695,-0.002046,0.000204,0.016543,-0.001005,-0.003831,-0.000056,-0.000540,-0.001805,0.001337,0.000263,0.002680,0.010431,-0.002079,-0.000448,0.001161,0.000963,-0.001673,-0.002434,-0.001108,0.001076,-0.001592,...,-0.082474,-0.056676,0.034234,0.083262,-0.141593,0.035778,-0.003047,0.103454,-0.002321,-0.008857,0.091990,0.040857,0.054758,-0.023202,-0.059826,0.030288,0.089759,0.003673,-0.005119,-0.049137,0.071554,0.005455,-0.004779,0.023652,0.027152,-0.004509,-0.004674,0.025301,-0.001915,0.012238,-0.092273,0.128352,-0.036696,-0.052496,0.000216,-0.017777,-0.071705,0.032427,0.0,0.0
1,0.000023,0.000205,0.000300,0.000830,0.001317,0.000176,0.000195,0.003127,-0.000270,0.135351,-0.066458,-0.002117,-0.008549,-0.000756,-0.001669,-0.000805,-0.000200,-0.003920,-0.001037,-0.000325,0.000305,0.001003,-0.001516,-0.002988,0.000205,-0.000863,-0.000431,0.007480,0.000126,0.003632,0.001763,-0.002241,0.000718,0.001416,-0.000325,-0.001306,-0.002652,-0.002690,0.004119,-0.000618,...,-0.067677,-0.106781,0.010305,0.074986,-0.177826,0.010226,-0.080694,0.059865,0.033146,0.014908,0.084805,0.033596,0.009942,0.041505,-0.054388,0.048819,0.130419,-0.004192,-0.034557,0.055314,0.078563,-0.069290,-0.094482,0.014324,0.050206,0.049959,0.009183,0.013774,-0.031543,0.016282,-0.120321,0.129150,0.021005,-0.035324,-0.039046,-0.070420,0.005137,-0.036460,0.0,0.0
2,0.000232,0.002136,0.001662,0.003528,0.002660,0.001212,0.001657,0.014204,-0.001141,0.672527,-0.333679,-0.015419,-0.044842,0.007236,-0.006178,0.004738,-0.004949,-0.016384,-0.008157,-0.005328,-0.001707,0.010427,-0.005253,-0.006108,-0.003040,-0.002782,-0.003031,-0.001039,-0.004362,-0.000904,0.023975,-0.002659,-0.000616,-0.001105,0.000848,-0.003174,0.000888,-0.000633,-0.003195,-0.002484,...,-0.009010,0.123925,-0.056231,-0.012941,0.020415,0.021919,0.017543,-0.010157,0.010535,-0.002024,-0.061336,-0.067615,0.045217,-0.007951,-0.047106,-0.004684,-0.024273,-0.001434,0.002793,-0.067825,0.023148,0.039794,-0.000913,0.026878,0.071711,0.000179,0.057456,-0.020544,-0.013863,-0.051055,-0.008946,0.040648,-0.024780,0.010070,0.005315,0.008530,-0.069809,0.042665,0.0,0.0
3,0.000062,0.032253,0.000955,0.001973,0.003118,0.000781,0.001006,0.007997,-0.000602,0.397574,-0.181950,-0.009445,-0.026125,-0.002921,-0.004466,-0.002667,-0.002619,-0.008445,-0.004755,-0.001404,-0.000262,-0.000495,-0.003113,-0.003810,0.001288,-0.001617,0.003926,0.000758,-0.001419,-0.002186,-0.002573,-0.002459,0.000819,0.010411,0.006836,-0.002790,-0.000401,-0.002067,-0.000690,-0.005267,...,-0.096941,0.013947,0.051218,0.101725,0.021286,-0.039433,0.126173,0.057232,-0.072166,0.007516,0.062219,-0.010036,0.046219,-0.114603,-0.032658,-0.002398,0.057861,0.019906,0.032772,-0.159197,0.018412,0.041492,0.072615,0.013135,0.027138,-0.031024,-0.016306,-0.014553,0.037170,-0.038833,0.006320,0.041956,-0.092880,-0.030048,0.052681,-0.044202,-0.157499,0.008607,0.0,0.0
4,0.000021,0.000122,0.000098,0.000315,0.000481,0.000059,0.000006,0.001480,-0.000186,0.053956,-0.026844,-0.001029,-0.002942,-0.000665,-0.000856,-0.000333,-0.000649,-0.001403,-0.000345,-0.000368,0.000211,0.000621,-0.001734,-0.001421,0.000960,-0.000619,0.000697,0.000146,-0.000065,0.001915,-0.001238,-0.001454,0.002561,0.001535,-0.000704,-0.002993,-0.003219,-0.002587,0.004631,-0.001809,...,-0.141311,-0.115561,0.008468,0.045653,-0.111014,-0.027988,-0.060813,0.068110,-0.048049,-0.011767,0.112474,0.041393,0.019384,0.199989,-0.001632,0.013736,0.092415,-0.031543,-0.136348,-0.056593,-0.079942,-0.029110,-0.204743,-0.033275,-0.010546,-0.074664,-0.057938,0.043697,-0.049172,-0.025471,-0.176209,0.

In [19]:
X_pred.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,0.000205,0.004119,0.001886,0.005541,0.004621,0.002209,0.002501,0.019721,-0.000720,0.832000,-0.409821,-0.019291,-0.054524,-0.005951,-0.006250,-0.002057,-0.005828,-0.019303,-0.010273,-0.007537,-0.001896,-0.001126,-0.004573,-0.007846,-0.005573,-0.001608,-0.002892,-0.001568,-0.003840,0.000618,-0.001385,-0.002487,-0.000099,-0.001320,-0.003596,0.001909,0.001231,-0.000752,-0.003661,0.001132,...,0.014721,0.030185,0.054354,-0.024916,-0.036835,-0.060842,-0.023411,0.069591,-0.005872,0.000404,0.011732,-0.024614,0.023996,-0.008352,-0.000310,-0.007912,0.009947,-0.007019,-0.050103,-0.028380,0.014146,0.012882,-0.015664,0.008190,0.016295,-0.020449,0.012703,-0.026614,-0.028795,0.037630,-0.001160,0.029760,0.017396,0.009345,0.004881,0.014675,0.045259,-0.013800,-0.010011,-0.021705
1,0.000795,0.003386,0.002407,0.004711,0.003789,0.001556,0.002151,0.018426,-0.000797,0.808724,-0.399913,-0.018661,-0.051842,-0.005176,-0.005169,-0.005383,-0.005840,-0.020238,-0.008467,-0.008630,-0.001202,-0.002347,-0.005697,-0.007442,-0.005638,-0.000047,-0.003436,-0.001380,-0.004103,0.001353,-0.001594,-0.002622,-0.000995,-0.001098,-0.003687,0.000438,0.001525,-0.000066,-0.004025,0.001231,...,0.020834,0.025071,0.041447,-0.037683,-0.051127,-0.068255,-0.029277,0.061040,-0.008246,-0.003343,0.008551,-0.018967,-0.011156,0.002469,-0.003676,-0.011636,0.031673,-0.013797,-0.037278,-0.017210,0.039152,0.008887,-0.009920,-0.018996,0.037491,-0.002789,-0.000171,-0.029503,-0.040110,0.017968,0.029526,0.005663,0.033291,-0.014039,0.031630,-0.015159,0.051373,-0.016010,-0.016157,-0.036429
2,0.001254,0.009051,0.002766,0.004780,0.004031,0.001705,0.002024,0.018044,-0.000687,0.783218,-0.386446,-0.017922,-0.046742,-0.004541,-0.005028,-0.005123,-0.005093,-0.020764,-0.006937,-0.007308,-0.001165,-0.001625,-0.003465,-0.006827,-0.005225,-0.000015,-0.003072,-0.000505,-0.005043,0.003071,0.000418,-0.001851,-0.000300,-0.001235,-0.001247,0.002211,0.002453,-0.001136,-0.001366,0.001049,...,0.031334,0.074365,0.015625,-0.023519,-0.017973,-0.069099,-0.031729,0.067095,-0.013429,-0.024423,0.056925,-0.052328,0.036610,-0.049939,0.012354,-0.011728,-0.013384,0.007051,-0.046867,-0.047752,0.048763,-0.001306,-0.029643,0.026116,0.002881,-0.036906,0.012748,-0.012009,-0.003511,0.056076,-0.002584,0.047496,0.044404,0.028137,0.002885,0.001565,0.047713,-0.007490,-0.009340,-0.045792
3,0.001139,0.003939,0.003054,0.004471,0.003676,0.001355,0.002085,0.019031,-0.000873,0.771243,-0.379669,-0.017833,-0.046612,-0.003253,-0.002927,-0.003077,-0.004255,-0.019104,-0.006567,-0.006680,-0.001351,-0.000576,0.000827,-0.005873,-0.005868,0.001349,-0.001941,0.001935,-0.004163,0.003695,0.000227,-0.002724,-0.000133,-0.001863,-0.000372,0.000648,0.004531,0.000102,-0.004226,0.001595,...,0.022914,0.038260,0.040911,-0.050891,-0.057854,-0.074545,-0.022565,0.084786,-0.012002,0.008452,0.032150,-0.035318,0.031466,-0.004904,0.008567,-0.011445,0.022372,-0.002732,-0.070979,-0.026503,0.045691,0.026958,-0.022262,0.021954,0.025650,-0.018341,0.003905,-0.033958,-0.060089,0.043045,0.005593,0.040884,0.030375,0.012391,0.012334,0.012444,0.061116,-0.025301,0.001421,-0.045219
4,0.000176,0.003986,0.003683,0.005340,0.004260,0.001957,0.001651,0.018016,-0.000933,0.735118,-0.361969,-0.016857,-0.047175,-0.004456,-0.004965,-0.003697,-0.004328,-0.018180,-0.009020,-0.007051,-0.001548,-0.000525,-0.002861,-0.006975,-0.004577,-0.001417,-0.003378,0.000133,-0.004099,0.001207,-0.001382,-0.001485,0.000171,-0.001578,-0.001574,0.000039,0.004134,-0.000301,-0.005821,0.000261,...,-0.004003,0.070229,0.034365,0.003379,-0.053539,-0.099027,-0.047673,0.017912,0.016103,0.004311,0.042081,0.044880,-0.058249,0.014498,-0.004837,-0.025924,-0.022380,-0.009857,-0.053731,0.028737,0.033908,-0.009907,-0.033584,-0.030055,0.035697,0.011619

In [20]:
data.shape, X_pred.shape 

((667104, 502), (7103, 500))

In [21]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

target = data.importance         
data.drop(['importance', 'doc_id'], axis=1, inplace=True)     # taking the training data features
                                 
# del data

new_data, X_test, new_target, y_test = train_test_split(data, target, test_size = 0.95, random_state = 32) # in this our main data is split into train and test

#del data, target
gc.collect()

11

In [22]:
X_train, X_test, y_train, y_test = train_test_split(new_data, new_target, test_size = 0.50, random_state = 32) # in this our main data is split into train and test

gc.collect()

X_train.shape



(16677, 500)

In [0]:
from sklearn.svm import SVC

In [26]:
model_svm = SVC()

model_svm.fit(X_train, y_train)
print(model_svm.score(X_test, y_test))

0.7653795419115001


In [27]:
pred = model_svm.predict(X_test)
print(f1_score(pred, y_test))

0.44551509139861134


In [0]:
from sklearn.model_selection import RandomizedSearchCV
import time
from scipy.stats import uniform


model = SVC()

# Create regularization penalty space
kernel = ['rbf', 'poly','linear']

# Create regularization hyperparameter distribution using uniform distribution
C = [20,30, 40, 100, 150, 200,300,500]

# Create hyperparameter options
hyperparameters = dict(C=C, kernel = kernel)

# Create randomized search 5-fold cross validation and 100 iterations
clf = RandomizedSearchCV(model, hyperparameters, random_state=42, n_iter=24, cv=2, verbose=1, n_jobs=-1, scoring = 'f1')

s = time.time()
best_model = clf.fit(X_train, y_train)
e = time.time()


Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [0]:

# View best hyperparameters
# print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
# print('Best C:', best_model.best_estimator_.get_params()['C'])
# print("Model Operation Completed in :",(e-s)/60,"min")

best_model.best_estimator_



In [0]:
pred = best_model.predict(X_test)
print(f1_score(pred, y_test))

In [0]:
#pred = best_model.predict(X_test)
print(accuracy_score(pred, y_test))